In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import timeit

# hyperparameters
batch_size = 64
block_size = 128
max_iters = 25000
eval_interval = 500
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 256
n_head = 8
n_layer = 8
dropout = 0.1

torch.manual_seed(1337)

# Load the text data
with open('tiny_story.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Tokenize the text into words
words = text.split()
vocab = sorted(set(words))
vocab_size = len(vocab)

# Create a mapping from words to integers
stoi = {word: i for i, word in enumerate(vocab)}
itos = {i: word for i, word in enumerate(vocab)}
encode = lambda s: [stoi[word] for word in s.split()]
decode = lambda l: ' '.join([itos[i] for i in l])

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

print(f"len(train_data) - {len(train_data)}")
print(f"len(val_data) - {len(val_data)}")

# data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class WordLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = WordLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
start_time = timeit.default_timer()

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        elapsed_time = timeit.default_timer() - start_time
        minutes, seconds = divmod(elapsed_time, 60)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, Elapsed time: {int(minutes)} minutes, {seconds:.6f} seconds")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=100)[0].tolist()))

len(train_data) - 899118
len(val_data) - 99902
18.668502 M parameters
step 0: train loss 10.2289, val loss 10.2324, Elapsed time: 0 minutes, 33.818332 seconds
step 500: train loss 5.1298, val loss 5.0903, Elapsed time: 2 minutes, 44.161362 seconds
step 1000: train loss 4.5443, val loss 4.5441, Elapsed time: 5 minutes, 0.244241 seconds
step 1500: train loss 4.2544, val loss 4.2801, Elapsed time: 7 minutes, 30.729283 seconds
step 2000: train loss 3.9924, val loss 4.0548, Elapsed time: 12 minutes, 55.439457 seconds
step 2500: train loss 3.7844, val loss 3.9121, Elapsed time: 17 minutes, 49.580814 seconds
step 3000: train loss 3.5909, val loss 3.7764, Elapsed time: 22 minutes, 58.019260 seconds
step 3500: train loss 3.4492, val loss 3.6787, Elapsed time: 26 minutes, 41.843974 seconds
step 4000: train loss 3.3097, val loss 3.6088, Elapsed time: 32 minutes, 26.344057 seconds
step 4500: train loss 3.1876, val loss 3.5586, Elapsed time: 35 minutes, 24.510722 seconds
step 5000: train loss 3.077

In [2]:
starting_text = encode("Once upon a time")
starting_context = torch.tensor([starting_text], dtype=torch.long, device=device)
print(decode(m.generate(starting_context, max_new_tokens=500)[0].tolist()))

Once upon a time there was a huge mouse who lived in the tree. It was the sun, the most interesting things the all day ever! One day the puddle was rough but no matter how hard it almost touched it with its huge wings. When the leaves stopped, the ground was rough to a bird. The bird said "Come on! Let's go and take care of it." So the ship approached the tree said and ventured back home. The puddle was very happy. It was safe and happy. The bird thanked it for its warning. Then it saw its special friend of the tree and smiled. Story end Story Start Once upon a time, there was a fat cat. The cat loved to go to a wide open store. The cat was tall and worked all day long. The cat liked to look for all the colors. At the end of the local kids saw a big box. They decided to explore the house and open it. The bird was gentle, so happy to Fin's music. But the cat didn't listen to them. So, the dog decided to wait. He danced and bounced joyfully even faster. They saw more and trees and trees.

In [3]:
torch.save(model.state_dict(), 'generated_model_25Kiters.pth')

In [ ]:
model = WordLanguageModel()
model.load_state_dict(torch.load('generated_model.pth'))